In [ ]:
#r "BoSSSpad.dll"
//  #r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedCavity_ConvergenceStudy");

Project name is set to 'HeatedCavity_ConvergenceStudy'.
Opening existing database '\\hpccluster\hpccluster-scratch\gutierrez\HeatedCavity_ConvergenceStudy'.


In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedCavity_ConvergenceStudy");

In [ ]:
// Find number of cells used in the finest mesh
int NCellsFinestMesh = 0;
var allsess = myDb.Sessions;
foreach(var sess in allsess){
    int c = Convert.ToInt32(sess.KeysAndQueries["Grid:NoOfCells"]);
    if( c > NCellsFinestMesh)
    NCellsFinestMesh = c;
}

In [ ]:
int[] DGDegrees = new int[] {1,2,3,4} ;

In [ ]:
// int counter = 0;
// string[] varNames = new string[]{"VelocityX", "VelocityY", "Pressure","Temperature"};

// IEnumerable<KeyValuePair<string, double>>[] RegressionS = new IEnumerable<KeyValuePair<string, double>>[varNames.Length];

//     Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 2];
//     int cnt2 = -1;
//     for (int iCol = 0; iCol < 2; iCol++) {
//         for (int iRow = 0; iRow < 2; iRow++) {
//             var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
//             var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
//             PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle, };
//             cnt2++;
//             // BoSSS data
//             var fmt = new PlotFormat();
//             fmt.Style = Styles.Lines;
//             fmt.LineColor = allColors[cnt2 % allColors.Length];
//             fmt.PointType =  allPointTypes[cnt2 % allPointTypes.Length];
//             fmt.DashType = DashTypes.Solid;            
            
//             string dgDegStr = (varNames[cnt2] == "VelocityX"  || varNames[cnt2] == "VelocityY" ) ? "DGdegree:Velocity*":"DGdegree:"+varNames[cnt2];
//             var Tab = BoSSSshell.WorkflowMgm.SessionTable;
//             var Tab2 = Tab.ExtractColumns( dgDegStr,"Grid:hMin","Grid:NoOfCells" ,"L2Error_"+  varNames[cnt2]);
//             //Extract last point, the one against we are comparing our solutions with
//             Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh );
//             var plot = Tab2.ToPlot("Grid:hMin","L2Error_"+  varNames[cnt2],  dgDegStr);

//             int cc = 0;
//             foreach(var dataGroup in plot.dataGroups){
//                 dataGroup.Format.PointType = myPointTypes[cc];
//                 cc++;
//             }            
//             plot.LogX = true;
//             plot.LogY = true;

//             plot.Title = varNames[cnt2];
//             plot.TitleFont = 20;
//             plot.Xlabel = "h";  
//             plot.Ylabel = "L_2 Error";  
//             PlotTable[iCol, iRow] = plot;

//             RegressionS[cnt2] = plot.Regression();
//         }
//     }

In [ ]:
int counter = 0;
string[] varNames = new string[] { "VelocityX", "VelocityY", "Pressure", "Temperature" }; /////////////////////////////////////////////////////////////////

IEnumerable<KeyValuePair<string, double>>[] RegressionS = new IEnumerable<KeyValuePair<string, double>>[varNames.Length];

Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 2];
int cnt2 = -1;
for (int iCol = 0; iCol < 2; iCol++) {
    for (int iRow = 0; iRow < 2; iRow++) {
        var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
        var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
        PointTypes[] myPointTypes = new PointTypes[] { PointTypes.Diamond, PointTypes.Box, PointTypes.LowerTriangle, PointTypes.OpenLowerTriangle, };
        cnt2++;
        string varname = varNames[cnt2];

        var ExpPlotS = new List<Plot2Ddata>();

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines;
        fmt.LineColor = allColors[cnt2 % allColors.Length];
        fmt.PointType = allPointTypes[cnt2 % allPointTypes.Length];
        fmt.DashType = DashTypes.Solid;

        foreach (int pDeg in DGDegrees) {

            int actualDGDeg = varname == "Pressure" ? pDeg-1:pDeg;
            string dgDegStr = (varname == "VelocityX"  || varname == "VelocityY" ) ? "DGdegree:Velocity*":"DGdegree:"+varname;

            var pDegSessions = myDb.Sessions.Where(
                Si => (Convert.ToInt32(Si.KeysAndQueries[dgDegStr]) == actualDGDeg)
                ).ToArray();
            Plot2Ddata pDegPlot =
                pDegSessions.ToEstimatedGridConvergenceData(
                    varname,
                xAxis_Is_hOrDof: true, // false selects DOFs for x-axis 
                normType: varname == "Pressure" ? NormType.L2noMean_embedded : NormType.L2_embedded);
            ExpPlotS.Add(pDegPlot);
        }

        // ExpPlotS[0].dataGroups[0].Format.PointType = PointTypes.Diamond;
        // ExpPlotS[1].dataGroups[0].Format.PointType = PointTypes.Box;
        // ExpPlotS[2].dataGroups[0].Format.PointType = PointTypes.LowerTriangle;
        // ExpPlotS[3].dataGroups[0].Format.PointType = PointTypes.Asterisk;
        // ExpPlotS[0].dataGroups[0].Name = "$k = 1$";
        // ExpPlotS[1].dataGroups[0].Name = "$k = 2$";
        // ExpPlotS[2].dataGroups[0].Name = "$k = 3$";
        // ExpPlotS[3].dataGroups[0].Name = "$k = 4$";
        // foreach (var p in ExpPlotS) {
        //     p.dataGroups[0].Format.PointSize = 0.8;
        //     p.dataGroups[0].Format.DashType = DashTypes.Dotted;
        //     p.dataGroups[0].Format.LineWidth = 2;
        // }

        var ExpPlot = ExpPlotS [0]; // select 0-th object
        foreach (var p in ExpPlotS.Skip(1)) { // loop over other ( skip 0-th entry )
            ExpPlot = ExpPlot.Merge(p); // merge
        }
        
        ExpPlot.LogX = true;
        ExpPlot.LogY = true;

        ExpPlot.Title = varNames[cnt2];
        ExpPlot.TitleFont = 20;
        ExpPlot.Xlabel = "h";  
        ExpPlot.Ylabel = "L_2 Error";  
        PlotTable[iCol, iRow] = ExpPlot;
        RegressionS[cnt2] = ExpPlot.Regression();

    }
}



In [ ]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1000,yRes:1000)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityX 
 
 
 1 
 
 
 1 
 
 
 
 
 
 
 
 
 
 2 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityY 
 
 
 1 
 
 
 1 
 
 
 
 
 
 
 
 
 
 2 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Pressure 
 
 
 0 
 
 
 0 
 
 
 
 
 
 
 
 
 
 1 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Temperature 
 
 
 1 
 
 
 1 
 
 
 
 
 
 
 
 
 
 2 
 
 
 2

In [ ]:
int cnt = 0;
foreach(var reg in RegressionS){
    Console.WriteLine(varNames[cnt]);
    var regB = reg.ToArray();
    for(int j = 0; j < regB.Length; j++){       
        int dgDeg = ( regB[j].Key).Last() - '0';
        Console.Write("DG degree: "+ dgDeg+ " \t");
        double slope = regB[j].Value;
        Console.Write("Slope: " +slope + "\t");

        if((slope - (dgDeg + 1) ) >= -0.2)
        Console.WriteLine("Expected slope archieved");
        else
        Console.WriteLine("The slope value is too low");
    }
    cnt++;
}


VelocityX
DG degree: 1 	Slope: 0.8788781268701156	The slope value is too low
DG degree: 2 	Slope: 2.872532400906716	Expected slope archieved
VelocityY
DG degree: 1 	Slope: 0.87988624850685	The slope value is too low
DG degree: 2 	Slope: 2.6519212841355055	The slope value is too low
Pressure
DG degree: 0 	Slope: 1.0760098262654818	Expected slope archieved
DG degree: 1 	Slope: 1.9141081220873182	Expected slope archieved
Temperature
DG degree: 1 	Slope: 1.0265110714448025	The slope value is too low
DG degree: 2 	Slope: 2.876252875851804	Expected slope archieved
